#### To find dblp Profile Page for each person

In [1]:
import os
import json
import re
from copy import deepcopy

In [2]:
def fetch_purified_name(person_name):
    # eliminates text within brackets : Eg = changes Lawrence Page (Founder) to -> Lawrence Page
    # strips spaces
    # keeps only letters and spaces
    # changes name to lower case    
    
    final_name=""
    open_bracket=False
    for curr_ch in person_name:
        if curr_ch=='(':
            open_bracket=True
        elif curr_ch==')':
            open_bracket=False
        else:
            if not open_bracket:
                final_name+=curr_ch
    person_name=final_name.rstrip()
    person_name=person_name.lstrip()
    person_name = re.sub('[^a-zA-Z.\d\s]', '', person_name)
    person_name=person_name.lower()
    return person_name

In [3]:
def fetch_search_url(person_name):
    person_name=fetch_purified_name(person_name)
    person_name=person_name.replace(" ","%20")
    base_url=f"https://dblp.org/search/author/api?q={person_name}&format=json"
    #print(base_url)
    return base_url

In [4]:
def resolve_multiple_results(person_name, results_arr):
    
    # fetch person's purified name
    person_name=fetch_purified_name(person_name)
    
    # break purified name into tokens
    original_tokens=person_name.split(" ")
    num_original_tokens=len(original_tokens)
    
    
    extra_dict={}
    for idx, curr_result in enumerate(results_arr):
        his_name=fetch_purified_name(curr_result['info']['author'])
        
        found=0
        
        # tokenized name of candidate
        curr_tokens=his_name.split(' ')
        
        # check how many tokens in the original name are there in the candidate name
        for want_token in original_tokens:
            if want_token in curr_tokens:
                found+=1
                
        # if all tokens are there, then all is well
        if found==num_original_tokens:
            if len(curr_tokens) not in extra_dict:
                extra_dict[len(curr_tokens)]=[]
        
        
        # anyways store a dict with "key" as number of matched tokens and value as index of candidate
        extra_dict[len(curr_tokens)].append(idx)
        
    # check if it was possible for all tokens to get matched
    if num_original_tokens in extra_dict:
        # if there was JUST one such name
        if len(extra_dict[num_original_tokens])==1:
            return True, results_arr[extra_dict[num_original_tokens][0]]
    else:
        # it was not possible for all tokens to get matched, hence alias not found
        return False, results_arr     

In [5]:
test_name="ponnurangam kumaraguru"

In [6]:
import json,urllib.request
data = urllib.request.urlopen(fetch_search_url(test_name)).read()
output = json.loads(data)
print (output)

{'result': {'query': 'ponnurangam* kumaraguru*', 'status': {'@code': '200', 'text': 'OK'}, 'time': {'@unit': 'msecs', 'text': '0.86'}, 'completions': {'@total': '1', '@computed': '1', '@sent': '1', 'c': {'@sc': '1', '@dc': '1', '@oc': '1', '@id': '15284030', 'text': 'kumaraguru'}}, 'hits': {'@total': '1', '@computed': '1', '@sent': '1', '@first': '0', 'hit': [{'@score': '3', '@id': '1311788', 'info': {'author': 'Ponnurangam Kumaraguru', 'aliases': {'alias': 'Ponnurangam Kumarguru'}, 'url': 'https://dblp.org/pid/97/5147'}, 'url': 'URL#1311788'}]}}}


In [7]:
output['result']['hits']

{'@total': '1',
 '@computed': '1',
 '@sent': '1',
 '@first': '0',
 'hit': [{'@score': '3',
   '@id': '1311788',
   'info': {'author': 'Ponnurangam Kumaraguru',
    'aliases': {'alias': 'Ponnurangam Kumarguru'},
    'url': 'https://dblp.org/pid/97/5147'},
   'url': 'URL#1311788'}]}

In [8]:
!ls ..

create_mappings_to_other_databases  EXTRA_NAMES_LIST.json
dblp_code			    fetch_people_seed_from_wiki
ensure_consistency		    FINAL_NAMES_LIST.json
explore_wiki_data		    scrape_wiki


In [9]:
with open("../create_mappings_to_other_databases/dblp_mapping.json",'r') as fd:
    seeked_people_df=json.load(fd)

In [10]:
seeked_people_df[0]

{'ire_person_name': 'Abraham Silberschatz',
 'ire_wiki_id': 0,
 'DBLP author ID (P2456)': 's/AbrahamSilberschatz'}

In [11]:
property_name='DBLP author ID (P2456)'

In [12]:
error_cnt=0
first_aim=0
multiple_folks=0
heuristic_aim=0

In [13]:
file_name="dblp_search_results.json"

In [14]:
def fetch_trivial_url(dblp_id):
    template=f'https://dblp.org/pid/{dblp_id}.html'
    return template

In [15]:
def save_results():
    with open(file_name,'w') as fd:
        json.dump({'first_aim':first_aim,'error_cnt':error_cnt,'multiple_folks':multiple_folks,'resolved_multiple':heuristic_aim,'final_result':seeked_people_df}, fd, indent=1,default=str)

In [16]:
#multiple_results=[]
#result_arr=[]
done_idx=-1
for curr_obj in seeked_people_df:
    curr_name=curr_obj['ire_person_name']
    done_idx+=1
    curr_obj['url_use']=None
    if curr_obj[property_name]!=None:
        curr_obj['url_use']=fetch_trivial_url(curr_obj[property_name])
        first_aim+=1
    else:
        
        # fetch url for the search bar
        search_url=fetch_search_url(curr_name)
        final_data={'person_name':curr_name,'trivial':False}
        try:
            data = urllib.request.urlopen(search_url).read()
            output = json.loads(data)
            num_hits=len(output['result']['hits']['hit'])
            if num_hits!=1:
                #print("hits is ", num_hits)
                multiple_folks+=1
                # try to resolve
                final_data['multiple_results']=deepcopy(output['result']['hits']['hit'])
                verdict, hope_res=resolve_multiple_results(curr_name,output['result']['hits']['hit'])
                if verdict:
                    heuristic_aim+=1
                    final_data['data']=deepcopy(hope_res)
                    #print("resolved")
                # unresolved
                #multiple_results.append((num_hits, curr_name))

            else:
                # already perfect
                first_aim+=1
                final_data['trivial']=True
                final_data['data']=output['result']['hits']['hit'][0]
            #print(final_data)
            curr_obj['url_use']=final_data['data']['info']['url']
            #break

        except Exception as E:
            #print("exception is ",E)
            error_cnt+=1
            #print(json.dumps(output, indent=3))
            #errors.append((E, curr_name))
            final_data['error']=E
            final_data['error_result']=output
        curr_obj['search_result']=final_data
    if done_idx%100==0:
        #break
        print("Done idx is ", done_idx)
        save_results()
    #print("###############################")

Done idx is  0
Done idx is  100
Done idx is  200
Done idx is  300
Done idx is  400
Done idx is  500
Done idx is  600
Done idx is  700
Done idx is  800
Done idx is  900
Done idx is  1000
Done idx is  1100
Done idx is  1200
Done idx is  1300
Done idx is  1400
Done idx is  1500
Done idx is  1600
Done idx is  1700
Done idx is  1800
Done idx is  1900
Done idx is  2000
Done idx is  2100
Done idx is  2200
Done idx is  2300
Done idx is  2400
Done idx is  2500
Done idx is  2600
Done idx is  2700
Done idx is  2800
Done idx is  2900
Done idx is  3000
Done idx is  3100
Done idx is  3200
Done idx is  3300
Done idx is  3400


In [17]:
save_results()

In [18]:
!ls

dblp_search_person_page.ipynb  scrape_page_data.ipynb
dblp_search_results.json       trial_dblp_search_results.json
parsed_xmls_dblp.json	       trial_parsed_xmls_dblp.json
